In [ ]:
    sales_orders_schema = StructType(
    [
        StructField("customer_id", LongType(), True),
        StructField("customer_name", StringType(), True),
        StructField("order_datetime", StringType(), True),
        StructField("order_number", LongType(), True),
        StructField(
            "ordered_products",
            ArrayType(
                StructType(
                    [
                        StructField("curr", StringType(), True),
                        StructField("id", StringType(), True),
                        StructField("name", StringType(), True),
                        StructField("price", IntegerType(), True),
                        StructField("qty", IntegerType(), True),
                        StructField("unit", StringType(), True),
                        StructField(
                            "promotion_info",
                            StructType(
                                [
                                    StructField("promo_id", IntegerType(), True),
                                    StructField("promo_qty", IntegerType(), True),
                                    StructField("promo_disc", DecimalType(3, 2), True),
                                    StructField("promo_item", StringType(), True),
                                ]
                            ),
                            True,
                        ),
                    ]
                ),
                True,
            ),
            True,
        ),
        StructField("number_of_line_items", LongType(), True),
        StructField(
            "clicked_items", ArrayType(ArrayType(StringType(), True), True), True
        ),
        StructField(
            "promo_info",
            ArrayType(
                StructType(
                    [
                        StructField("promo_id", IntegerType(), True),
                        StructField("promo_qty", IntegerType(), True),
                        StructField("promo_disc", DecimalType(3, 2), True),
                        StructField("promo_item", StringType(), True),
                    ]
                ),
                True,
            ),
            True,
        ),
    ]
) 
@dlt.table(
    comment="Load data to sales_orders cleansed table",
    table_properties={"pipelines.reset.allowed": "true"},
    spark_conf={"pipelines.trigger.interval": "60 seconds"},
    temporary=False,
)
def sales_orders_cleansed():
    return (
        dlt.readStream("sales_orders_stream_raw")        
        .select(get_json_object(to_json(col("payload")), "$.after").alias("row"))
        .withColumn("row", regexp_replace("row", '"\\[', "["))
        .withColumn("row", regexp_replace("row", '\\]"', "]"))
        .withColumn("row", regexp_replace("row", "\\\\", ""))
        .select(from_json(col("row"), sales_orders_schema).alias("row"))
        .select("row.*")
        .withColumn("ordered_products", explode("ordered_products"))
        .withColumn("order_datetime", from_unixtime("order_datetime"))
        .withColumn("product_id", col("ordered_products").id)
        .withColumn("unit_price", col("ordered_products").price)
        .withColumn("quantity", col("ordered_products").qty)
    )

In [ ]:
   schema = ArrayType(
    StructType(
        [
            StructField("qty", IntegerType(), True),
            StructField("unit", StringType(), True),
            StructField("curr", StringType(), True),
            StructField("id", StringType(), True),
            StructField("name", StringType(), True),
            StructField("price", IntegerType(), True),
            StructField(
                "promotion_info",
                StructType(
                    [
                        StructField("promo_id", IntegerType(), True),
                        StructField("promo_qty", IntegerType(), True),
                        StructField("promo_disc", DecimalType(3, 2), True),
                        StructField("promo_item", StringType(), True),
                    ]
                ),
                True,
            ),
        ]
    )
)

schema2 = ArrayType(
    StructType(
        [
            StructField("promo_id", IntegerType(), True),
            StructField("promo_qty", IntegerType(), True),
            StructField("promo_disc", DecimalType(3, 2), True),
            StructField("promo_item", StringType(), True),
        ]
    ),
    True,
)
schema3 = ArrayType(ArrayType(StringType(), True), True)

@dlt.table(
     comment="Load data to customers cleansed table",
     table_properties={"pipelines.reset.allowed": "true"},
     spark_conf={"pipelines.trigger.interval": "60 seconds"},
     temporary=False,
)

def sales_orders_batch_cleansed():
    return (
        dlt.read("sales_orders_batch_raw")
        .select("*")
        .withColumn("clicked_items", from_json(col("clicked_items"), schema3))
        .withColumn("promo_info", from_json(col("promo_info"), schema2))
        .withColumn("ordered_products", from_json(col("ordered_products"), schema))
        .withColumn("ordered_products", explode("ordered_products"))
        .withColumn("order_datetime", from_unixtime("order_datetime"))
        .withColumn("product_id", col("ordered_products").id)
        .withColumn("unit_price", col("ordered_products").price)
        .withColumn("quantity", col("ordered_products").qty)
    )
    
def customers_cleansed():
    return (
        dlt.read("customers_raw")
    )

In [ ]:
   @dlt.table(
     comment="Load data to a products cleansed table",
     table_properties={"pipelines.reset.allowed": "true"},
     spark_conf={"pipelines.trigger.interval": "60 seconds"},
     temporary=False,
)

In [ ]:
import dlt
from pyspark.sql.functions import col, expr

@dlt.view(spark_conf={"pipelines.incompatibleViewCheck.enabled": "false"})
def users():
  return spark.readStream.format("delta").option("ignoreChanges", "true").table("retail_org.sales_orders_batch_cleansed")

dlt.create_streaming_live_table("sales_orders_cleansed")

dlt.apply_changes(
  target = "sales_orders_cleansed",
  source = "users",
  keys = ["order_number"],
  sequence_by = col("order_datetime"),
  stored_as_scd_type = 1
)

In [ ]:
%sql
select * from retail_org.sales_orders_cleansed